In [ ]:
! pip install segmentation-models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import pandas as pd
import os.path

from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage import exposure
from skimage.filters import sobel
from sklearn.metrics import jaccard_score
import random
import time

from keras.models import *
from keras.layers import *
import tensorflow as tf
from keras.optimizers import *
import keras.backend as K
from random import shuffle

from segmentation_models import Unet
from segmentation_models import get_preprocessing

In [ ]:
def smooth(img):
    return 0.5*img + 0.5*(np.roll(img, +1, axis=0) + np.roll(img, -1, axis=0) +
        np.roll(img, +1, axis=1) + np.roll(img, -1, axis=1) )

def returnIndex(a , value):
    k = np.size(a)
    for i in range(k):
        if(a[i]==value):
            return i
        
def duplex_segmentation(img_path):
    area_frac_duplex=[]
    duplex_image_id=[]
    image = Image.open(img_path).convert('F')
    image = np.copy(np.reshape(np.array(image), image.size[::-1])/255.)   
    image = exposure.equalize_adapthist(image, clip_limit=8.3)
    image = (smooth(smooth(image)))
    image_copy = image
    markers = np.zeros_like(image)
    markers[image > np.median(image) - 0.10*np.std(image)] = 1     
    markers[image < np.median(image) - 0.10*np.std(image)] = 2
    fig, (ax1) = plt.subplots(1, sharex=True, sharey=True)
    elevation_map = sobel(image)
    segmentation = morphology.watershed(elevation_map, markers)
    segmentation = ndi.binary_fill_holes(segmentation - 1)
    labeled_grains, _ = ndi.label(segmentation)
    return np.array(segmentation)

def lamellar_segmentation(img_path):
    s = 604
    PIL_image = Image.open(img_path)
    img_in = np.array(PIL_image.convert('L'))
    img_in = cv2.resize(img_in, (s,s), interpolation=cv2.INTER_CUBIC)
    img_in = cv2.blur(img_in, (3,3))
    img_in = cv2.medianBlur(img_in, 3)
    thi, img_in = cv2.threshold(img_in,np.median(img_in),255,cv2.THRESH_BINARY)
    return np.array(img_in)

def read_equi_data(img_mask_path):
    PIL_image = Image.open(img_mask_path[0])
    img_in = np.array(PIL_image.convert('L'))
    img_in = cv2.resize(img_in, (s,s), interpolation=cv2.INTER_CUBIC)
    thi, img_in = cv2.threshold(img_in,np.median(img_in),255,cv2.THRESH_BINARY)
    img_in = 255 - img_in
    img_in = cv2.medianBlur(img_in, 3)
    img_in = np.reshape(img_in, (s,s,1))
    PIL_mask = Image.open(img_mask_path[1])
    mask_in = np.array(PIL_mask.convert('L'))
    mask_in = cv2.resize(mask_in, (s,s), interpolation=cv2.INTER_CUBIC)
    thi, mask_in = cv2.threshold(mask_in,np.median(mask_in),255,cv2.THRESH_BINARY)
    mask_in = mask_in/255
    mask_in = mask_in*2
    mask_in = np.reshape(mask_in, (s,s,1))
    mask_in = tf.keras.utils.to_categorical(mask_in, num_classes = 3, dtype ="uint8")
    return (img_in, mask_in)

def read_lam_data(img_mask_path):
    PIL_image = Image.open(img_mask_path[0])
    img_in = np.array(PIL_image.convert('L'))
    img_in = cv2.resize(img_in, (s,s), interpolation=cv2.INTER_CUBIC)
    img_in = cv2.blur(img_in, (3,3))
    img_in = cv2.medianBlur(img_in, 3)
    img_in = np.reshape(img_in, (s,s,1))
    PIL_mask = Image.open(img_mask_path[1])
    mask_in = np.array(PIL_mask.convert('L'))
    mask_in = cv2.resize(mask_in, (s,s), interpolation=cv2.INTER_CUBIC)
    thi, mask_in = cv2.threshold(mask_in,np.median(mask_in),255,cv2.THRESH_BINARY)
    mask_in = mask_in/255
    mask_in = np.reshape(mask_in, (s,s,1))
    mask_in = tf.keras.utils.to_categorical(mask_in, num_classes = 3, dtype ="uint8")
    return (img_in, mask_in)

In [ ]:
num_equi_training_samples = 10
for i in range(1, num_equi_training_samples + 1):
    file_id = str(num)
    img_path = 'equi_images/image_' + file_id + '.png'
    res = duplex_segmentation(img_path)
    res_for_cv2 = (res*255.).astype('uint8')
    outfile = 'equi_masks/mask_' + file_id + '.png'
    cv2.imwrite(outfile, res_for_cv2)

In [ ]:
num_lam_training_samples = 10
for num in range(1, num_lam_training_samples + 1):
    file_id = str(num)
    img_path = 'lam_images/image_' + file_id + '.png'
    res = lamellar_segmentation(img_path)
    res_for_cv2 = res #(res*255.).astype('uint8')
    outfile = 'lam_masks/mask_' + file_id + '.png'
    cv2.imwrite(outfile, res_for_cv2)

In [ ]:
train_equi_mask_paths = [("equi_images/image_%d.png"%i,
                     "equi_masks/mask_%d.png"%i) for i in range(1,num_equi_training_samples + 1)] 
            
    
train_lam_mask_paths = [("lam_images/image_%d.png"%i,
                     "lam_masks/mask_%d.png"%i) for i in range(1,num_lam_training_samples + 1)] 


train_lam_paths = train_lam_mask_paths[int(1):]
train_equi_paths = train_equi_mask_paths[int(1):]
num_train_images = len(train_lam_paths) + len(train_equi_paths)
num_test_images = len(test_lam_paths) + len(test_equi_paths)

In [ ]:
out_imgs = np.zeros((num_train_images,)+(s,s)+(1,)) 
out_masks = np.zeros((num_train_images,)+(s,s)+(3,)) 
for i, img_mask_path in enumerate(train_lam_paths):
    img, mask = read_lam_data(img_mask_path) # import images, save as array, crop and resize the image to (256,256,1)
    out_imgs[i,...] = img # create single array of images
    out_masks[i,...] = mask # create single array of masks

flag = len(train_lam_paths)
    
for i, img_mask_path in enumerate(train_equi_paths):
    img, mask = read_equi_data(img_mask_path) # import images, save as #array, crop and resize the image to (256,256,1)
    out_imgs[i+flag,...] = img # create single array of images
    out_masks[i+flag,...] = mask # create single array of masks

In [ ]:
BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)

x_train = preprocess_input(out_imgs)

N = 1

base_model = Unet(BACKBONE, classes = 3, activation='softmax', encoder_weights='imagenet')

inp = Input(shape=(None, None, N))
l1 = Conv2D(3, (1, 1))(inp) 
out = base_model(l1)
#out = BatchNormalization()(out)

model = Model(inp, out, name=base_model.name)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
steps = 0
overall_acc = []
overall_iou = []

out_imgs = np.zeros((num_train_images,)+(s,s)+(1,)) # define the input images for the model
out_masks = np.zeros((num_train_images,)+(s,s)+(3,)) # define the input masks for the model
for i, img_mask_path in enumerate(train_lam_paths):
    img, mask = read_lam_data(img_mask_path) # import images, save as array, crop and resize the image to (256,256,1)
    out_imgs[i,...] = img # create single array of images
    out_masks[i,...] = mask # create single array of masks

flag = len(train_lam_paths)
    
for i, img_mask_path in enumerate(train_equi_paths):
    img, mask = read_equi_data(img_mask_path) # import images, save as #array, crop and resize the image to (256,256,1)
    out_imgs[i+flag,...] = img # create single array of images
    out_masks[i+flag,...] = mask # create single array of masks
        
BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)
x_train = preprocess_input(out_imgs)

model.fit(x=x_train, y=out_masks, batch_size=3, epochs=5, verbose = 1)

steps += 20

test_equi_mask_paths = [("equi_images/image_%d.png"%i,
                     "equi_masks/mask_%d.png"%i) for i in range(num_equi_training_samples + 1, num_equi_training_samples + 11)] 
                
test_lam_mask_paths = [("lam_images/image_%d.png"%i,
                     "lam_masks/mask_%d.png"%i) for i in range(num_lam_training_samples + 1, num_lam_training_samples + 11)] 
    
num_test_images = len(test_lam_mask_paths) + len(test_equi_mask_paths)
    
out_imgs = np.zeros((num_test_images,)+(s,s)+(1,))
out_masks = np.zeros((num_test_images,)+(s,s)+(3,))
for i, img_mask_path in enumerate(test_lam_mask_paths):
    img, mask = read_lam_data(img_mask_path)
    out_imgs[i,...] = img
    out_masks[i,...] = mask
    
flag = len(test_lam_mask_paths)

for i, img_mask_path in enumerate(test_equi_mask_paths):
    img, mask = read_equi_data(img_mask_path)
    out_imgs[i+flag,...] = img
    out_masks[i+flag,...] = mask

    
prediction = model.predict_on_batch(out_imgs)
curr_round_acc = []
curr_round_iou = []
for i in range(len(test_lam_mask_paths)):
    img2 = Image.open(test_lam_mask_paths[i][1])
    img2 = np.array(img2.convert('L'))
    img2 = cv2.resize(img2, (256,256), interpolation=cv2.INTER_CUBIC)
    thi, img2 = cv2.threshold(img2, np.median(img2),255,cv2.THRESH_BINARY)
    img2 = img2/255
    img2 = img2.astype('uint8')
    img_pred = prediction[i][:,:,1]
    img_pred = cv2.resize(img_pred, (256,256), interpolation=cv2.INTER_CUBIC)
    thi, img_pred = cv2.threshold(img_pred, np.median(img_pred),255,cv2.THRESH_BINARY)
    img_pred = img_pred/255
    img_pred = img_pred.astype('uint8')
        
    iou = jaccard_score(img2, img_pred, average='weighted')
    curr_round_iou.append(iou)
    diff = (img2 == img_pred).astype('uint8')
    accuracy = np.sum(diff)/(256*256)
    curr_round_acc.append(accuracy)
        
for i in range(len(test_lam_mask_paths), num_test_images):
    img2 = Image.open(test_dup_mask_paths[i- len(test_lam_mask_paths)][1])
    img2 = np.array(img2.convert('L'))
    img2 = cv2.resize(img2, (256,256), interpolation=cv2.INTER_CUBIC)
    thi, img2 = cv2.threshold(img2, np.median(img2),255,cv2.THRESH_BINARY)
    img2 = img2/255
    img2 = img2.astype('uint8')
    img_pred = prediction[i][:,:,1]
    img_pred = cv2.resize(img_pred, (256,256), interpolation=cv2.INTER_CUBIC)
    thi, img_pred = cv2.threshold(img_pred, np.median(img_pred),255,cv2.THRESH_BINARY)
    img_pred = img_pred/255
    img_pred = img_pred.astype('uint8')
        
    iou = jaccard_score(img2, img_pred, average='weighted')
    curr_round_iou.append(iou)
    diff = (img2 == img_pred).astype('uint8')
    accuracy = np.sum(diff)/(256*256)
    curr_round_acc.append(accuracy)
        
overall_acc.append(curr_round_acc)
overall_iou.append(curr_round_iou)